In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import nltk
import joblib
import random


In [27]:
df = pd.read_csv('../../HO1/data/coursera_course_details_preprocessed.csv')
df

,Title,Partner,Subject,Description,Rating,Level,Duration (in hour),Skills,URL
0,microsoft business analyst professional certif...,Microsoft,Business Strategy,launch career business analyst build jobready ...,4.6,Beginner level,120.0,business process modeling pivot table chart bu...,https://www.coursera.org/professional-certific...
1,data engineering foundation specialization,IBM,Data Management,build foundation data engineering career devel...,4.7,Beginner level,80.0,database design database management system big...,https://www.coursera.org/specializations/data-...
2,psychological first aid,Johns Hopkins University,Psychology,psychological first aid,4.8,Beginner level,6.0,trauma care crisis intervention triage stress ...,https://www.coursera.org/learn/psychological-f...
3,tableau business intelligence analyst professi...,Tableau Learning Partner,Data Analysis,launch career data analytics build indemand sk...,4.7,Beginner level,320.0,spatial data analysis data visualization softw...,https://www.coursera.org/professional-certific...
4,think like cfo specialization,IESE Business School,Finance,think like cfo specialization,4.8,Beginner level,40.0,balance sheet working capital capital budgetin...,https://www.coursera.org/specializations/think...
...,...,...,...,...,...,...,...,...,...
355,ibm business analyst professional certificate,IBM,Business Essentials,get jobready business analyst build essential ...,4.8,Beginner level,120.0,stakeholder engagement business analytics busi...,https://www.coursera.org/professional-certific...
356,adobe content creator professional certificate,Adobe,Marketing,launch career content creator build indemand s...,4.7,Beginner level,160.0,storytelling content marketing social medium i...,https://www.coursera.org/professional-certific...
357,google automation python professional certificate,Google,Support and Operations,learn indemand skill like python git automatio...,4.8,Advanced level,240.0,unit testing programming principle github inte...,https://www.coursera.org/professional-certific...
358,generative ai engineering llm specialization,IBM,Machine Learning,advance ml career gen ai llm master essential ...,4.5,Intermediate level,120.0,text mining large language modeling applied ma...,https://www.coursera.org/specializations/gener...


In [28]:
df['CombinedText'] = (
    df['Title'].fillna('') + ' ' +
    df['Description'].fillna('') + ' ' +
    df['Skills'].fillna('')
)

In [29]:
df['CombinedText'][0]

'microsoft business analyst professional certificate launch career business analyst build jobready skill indemand career business analysis little month prior experience required get started business process modeling pivot table chart business analysis excel formula requirement analysis scrum software development user story requirement elicitation microsoft excel'

In [30]:
nltk.download('punkt')

def tokenize_text(text):
    return text.lower().split()

def average_vector(tokens, model, vector_size):
    valid_tokens = [model.wv[word] for word in tokens if word in model.wv]
    if not valid_tokens:
        return np.zeros(vector_size)
    return np.mean(valid_tokens, axis=0)

[nltk_data] Downloading package punkt to C:\Users\Dwi Wahyu
[nltk_data]     Lestari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
# Vectorization using Word2Vec
df['Tokens'] = df['CombinedText'].apply(tokenize_text)
vector = Word2Vec(sentences=df['Tokens'], vector_size=100, window=5, min_count=2, workers=4, seed=42)
doc_vectors = np.array([average_vector(tokens, vector, 100) for tokens in df['Tokens']])


In [32]:
vectorized_df = pd.DataFrame(doc_vectors)
vectorized_df.to_csv("result/vectorized_courses_2.csv", index=False)

In [33]:
vectorized_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.032053,0.019989,-0.057976,0.131741,-0.281574,0.169285,0.088599,0.197511,-0.192316,-0.014281,...,-0.379899,0.372225,0.031647,-0.183440,0.095369,-0.166495,0.047119,0.181650,-0.142004,-0.087820
1,0.039411,0.023139,-0.060046,0.143107,-0.311720,0.187401,0.097163,0.220753,-0.214768,-0.014873,...,-0.418395,0.414339,0.035405,-0.204868,0.108329,-0.186016,0.048574,0.200645,-0.159994,-0.098370
2,0.015509,0.007087,-0.024232,0.055372,-0.119773,0.072363,0.038225,0.085683,-0.082214,-0.002606,...,-0.160208,0.159157,0.012214,-0.078965,0.040763,-0.070875,0.018669,0.076269,-0.060701,-0.035576
3,0.032721,0.022663,-0.058971,0.134695,-0.293386,0.177105,0.091002,0.207009,-0.200620,-0.016398,...,-0.394835,0.388537,0.033762,-0.191323,0.100558,-0.174311,0.046898,0.188483,-0.147927,-0.093323
4,0.028416,0.013724,-0.040366,0.100874,-0.212128,0.129399,0.071126,0.151687,-0.147797,-0.009029,...,-0.290693,0.285133,0.021445,-0.140871,0.077810,-0.127896,0.035187,0.139590,-0.107139,-0.067013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.034090,0.020203,-0.056420,0.130064,-0.280485,0.169259,0.088468,0.196819,-0.192727,-0.011908,...,-0.378065,0.372127,0.032316,-0.183242,0.093847,-0.166052,0.047481,0.182123,-0.144633,-0.086857
356,0.026668,0.017508,-0.046807,0.103573,-0.227862,0.135564,0.069183,0.158634,-0.156126,-0.009404,...,-0.305540,0.299350,0.026156,-0.146679,0.075960,-0.131736,0.038133,0.146787,-0.116092,-0.070872
357,0.031702,0.018887,-0.052547,0.120141,-0.258980,0.154989,0.083447,0.180177,-0.176466,-0.010277,...,-0.347110,0.343462,0.030078,-0.166873,0.087208,-0.153422,0.045213,0.166232,-0.132516,-0.079660
358,0.034452,0.017274,-0.050904,0.118901,-0.253505,0.151075,0.082352,0.179542,-0.175072,-0.009362,...,-0.341359,0.336892,0.028417,-0.166471,0.088073,-0.149619,0.041312,0.164343,-0.130223,-0.082387


In [34]:
# Cosine similarity
similarity_matrix = cosine_similarity(doc_vectors)

In [35]:
similarity_df = pd.DataFrame(similarity_matrix, index=df['Title'], columns=df['Title'])
similarity_df.to_csv("result/similarity_matrix_2.csv")

In [ ]:
bundle = {
    "word2vec_model": vector,
    "document_vectors": doc_vectors,
    "similarity_matrix": similarity_matrix,
    "dataframe": df
}

joblib.dump(bundle, "../models/course_recommender2.joblib")


['../models/course_recommender2.joblib']

In [37]:
def generate_top_recommendations(df, similarity_matrix, top_n=5):
    recommendations = []
    for idx, row in df.iterrows():
        sim_scores = list(enumerate(similarity_matrix[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_indices = [i for i, score in sim_scores[1:top_n+1]]
        recommended_titles = df.iloc[top_indices]['Title'].tolist()
        recommendations.append(recommended_titles)
    return recommendations

In [38]:
df['TopRecommendations'] = generate_top_recommendations(df, similarity_matrix, top_n=5)
df[['Title', 'TopRecommendations']].to_csv("result/top_recommendations_2.csv", index=False)

sample_idx = random.randint(0, len(df) - 1)
print(f"\nExample course: {df.iloc[sample_idx]['Title']}")
print("Top recommendations:")
for title in df.iloc[sample_idx]['TopRecommendations']:
    print(f" - {title}")


Example course: ibm project manager professional certificate
Top recommendations:
 - ibm program manager professional certificate
 - microsoft project management professional certificate
 - ibm project manager professional certificate
 - google project management professional certificate
 - generative ai project manager specialization
